In [98]:
import json, folium
import pandas as pd
import numpy as np
import re

In [99]:
df = pd.read_csv('data/경기도도시공원정보표준데이터.csv', encoding='euc-kr')

In [100]:
filename = 'data/경기도_geo_simple.json'
geo_data = json.load(open(filename,encoding='utf-8'))

In [101]:
df = df[['공원명','소재지지번주소','위도','경도','공원면적']]
df

,공원명,소재지지번주소,위도,경도,공원면적
0,꿈나무공원,경기도 이천시 중리동 15,37.278561,127.438819,2526.0
1,남천공원,경기도 이천시 중리동 210,37.278010,127.444436,3830.0
2,향교공원,경기도 이천시 창전동 408,37.284902,127.437589,1757.0
3,달맞이공원,경기도 이천시 창전동 423,37.285854,127.443215,1650.0
4,체육공원,경기도 이천시 창전동 432-10,37.285969,127.445970,1300.0
...,...,...,...,...,...
494,영통중앙,경기도 수원시 영통구 영통동961-1,37.257187,127.072059,137062.5
495,반달,경기도 수원시 영통구 영통동 1012-4,37.250874,127.078858,16015.3
496,살구골,경기도 수원시 영통구 영통동 989-3,37.250663,127.071687,11753.7
497,벽적골,경기도 수원시 영통구 영통동 969-2,37.248678,127.061893,13502.4


In [102]:
pop = pd.read_csv('../02.Pandas/data/주민등록인구집계현황.csv', encoding='euc-kr')
pop = pop[['행정구역구분명','행정구역명','총 인구수']]
pop = pop[pop.행정구역구분명.str.contains('시군') | pop.행정구역구분명.str.contains('구')]

In [103]:
gu_city = ['수원시', '고양시', '용인시', '성남시', '안산시', '안양시']
pop.행정구역명 = pop.행정구역명.str.strip()
drop_index = []
for index in pop[pop.행정구역구분명 == '시군'].index:
    if pop.행정구역명[index].split()[-1] in gu_city:
        drop_index.append(index)
pop.drop(drop_index, inplace=True)
pop.head()

,행정구역구분명,행정구역명,총 인구수
1,시군,경기도 가평군,62197
9,구,경기도 고양시 덕양구,487874
31,구,경기도 고양시 일산동구,296590
44,구,경기도 고양시 일산서구,290738
56,시군,경기도 과천시,77775


In [104]:
pop['시군구'] = pop.행정구역명.apply(lambda x: ' '.join(x.split()[1:]))
pop.head()

,행정구역구분명,행정구역명,총 인구수,시군구
1,시군,경기도 가평군,62197,가평군
9,구,경기도 고양시 덕양구,487874,고양시 덕양구
31,구,경기도 고양시 일산동구,296590,고양시 일산동구
44,구,경기도 고양시 일산서구,290738,고양시 일산서구
56,시군,경기도 과천시,77775,과천시


In [105]:
pop['시군구'] = pop.행정구역명.apply(lambda x: ' '.join(x.split()[1:]))
pop.head()

,행정구역구분명,행정구역명,총 인구수,시군구
1,시군,경기도 가평군,62197,가평군
9,구,경기도 고양시 덕양구,487874,고양시 덕양구
31,구,경기도 고양시 일산동구,296590,고양시 일산동구
44,구,경기도 고양시 일산서구,290738,고양시 일산서구
56,시군,경기도 과천시,77775,과천시


In [106]:
df[['시군구2']] = df.소재지지번주소.apply(lambda x: 
                    x.split()[1] if x.split()[2][-1] != '구' else ' '.join(x.split()[1:3]))
df.tail()

AttributeError: 'float' object has no attribute 'split'

In [ ]:
map = folium.Map(location=[37.4,127.1], zoom_start=9, tiles='Stamen Toner')
folium.Choropleth(
    geo_data=geo_data,
    data=df.구별면적비율,        #단계구분도로 보여줄 데이터
    columns=[df.index,df.구별면적비율],      # 데이터 프레임에서 추출할 column
    fill_color='YlOrRd',       # Color map
    key_on='properties.name'
).add_to(map)
map